In [6]:
from collections import Counter
import numpy as np
import re 

def normalize_answer(s):
    # Lower case, remove punctuation, and whitespaces ### PLEASE CHECK If logic correct / desired ######## 
    return ' '.join(re.sub(r'[^A-Za-z0-9]', ' ', re.sub(r'\b(a|an|the)\b', ' ', s.lower())).split())

def metrics(annotation_answers, rag_answers):
    f1_scores, exact_matches, recall_scores = [], [], []
    
    for pred, truth in zip(annotation_answers, rag_answers):
        pred_tokens, truth_tokens = normalize_answer(pred).split(), normalize_answer(truth).split()
        common_tokens = Counter(pred_tokens) & Counter(truth_tokens)
        num_common = sum(common_tokens.values())
        
        prec = (num_common / len(pred_tokens))
        rec = (num_common / len(truth_tokens)) 

        f1_scores.append( 0 if prec+rec == 0 else (2*(prec*rec)/(prec + rec)))
        exact_matches.append(int(pred_tokens == truth_tokens))
        recall_scores.append(rec if truth_tokens else 0)
    
    return np.mean(f1_scores), np.mean(exact_matches), np.mean(recall_scores)

In [48]:
"""
Citation: 
    The use of chatgpt for the normalize answer and for using the counter function only
"""

(0.5, 0.0, 0.5)


In [1]:
def getFileData(filename):
    f = open(filename, "r")
    data = f.readlines()
    f.close()
    data = [i.strip() for i in data]
    return data

In [10]:
refrence_answers = getFileData("SubmissionData/test/reference_answers.txt")
system_generated_answers = getFileData("SubmissionData/system_outputs/system_output_1.txt")



In [11]:
# GPT4All Embeddings

metrics(refrence_answers, system_generated_answers)

(0.2303667203579527, 0.0, 0.1764070797818063)

In [12]:
refrence_answers = getFileData("SubmissionData/test/reference_answers.txt")
system_generated_answers = getFileData("SubmissionData/system_outputs/system_output_2.txt")



In [13]:
# all-miniLM-l6-v2 embeddings

metrics(refrence_answers, system_generated_answers)

(0.35622910840011396, 0.0379746835443038, 0.3376383147944196)